In [43]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import csv
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import SGDClassifier

In [44]:
readIn = pd.read_csv("data/df_dummy_prepped-2-2.csv")
df = pd.DataFrame(readIn)

In [45]:
readIn = pd.read_csv("data/training_data.csv")
df_og = pd.DataFrame(readIn)


In [46]:
cols_to_keep=['CONTRACT_ID','BORROWER_ID', 'CONTRACT_CREDIT_INTERMEDIARY']
df_og=df_og[cols_to_keep]

In [47]:
df_og.shape

(1602753, 3)

In [48]:
df.shape

(1602753, 490)

In [49]:
merged_df = pd.merge(df, df_og, on=['CONTRACT_ID', 'BORROWER_ID'], how='left')
merged_df.isna().sum()

CONTRACT_ID                                         0
BORROWER_ID                                         0
CONTRACT_CREDIT_LOSS                                0
CONTRACT_DATE_OF_LOAN_AGREEMENT                     0
CONTRACT_DEPT_SERVICE_TO_INCOME                     0
                                                ...  
CONTRACT_TYPE_OF_INTEREST_REPAYMENT_110001.0        0
CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140001.0        0
CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140002.0        0
CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140003.0        0
CONTRACT_CREDIT_INTERMEDIARY                    59422
Length: 491, dtype: int64

In [50]:
del df
del df_og

In [51]:
input=list(merged_df.columns)

In [52]:
merged_df ['CONTRACT_CREDIT_INTERMEDIARY'].value_counts()
# merged_df ['CONTRACT_TYPE_OF_INTEREST_REPAYMENT'].isna().sum()

CONTRACT_CREDIT_INTERMEDIARY
2.0        1069644
1.0         454073
20001.0      18654
0.0            960
Name: count, dtype: int64

In [53]:
to_remove=['CONTRACT_ID', 'BORROWER_ID', 'CONTRACT_DATE_OF_LOAN_AGREEMENT','CONTRACT_MATURITY_DATE', 'TARGET_EVENT_DAY', 'TARGET_EVENT']

In [54]:
new_input = [x for x in input if x not in to_remove]

In [55]:
new_input.remove('CONTRACT_CREDIT_INTERMEDIARY_0.0')
new_input.remove('CONTRACT_CREDIT_INTERMEDIARY_1.0')
new_input.remove('CONTRACT_CREDIT_INTERMEDIARY_2.0')
new_input.remove('CONTRACT_CREDIT_INTERMEDIARY_20001.0')

In [56]:
class_df=merged_df.dropna(subset=['CONTRACT_CREDIT_INTERMEDIARY'], axis=0)

In [57]:
target= 'CONTRACT_CREDIT_INTERMEDIARY'

In [58]:
new_input.remove('CONTRACT_CREDIT_INTERMEDIARY')

In [59]:
X = class_df[new_input]
y = class_df[target]

In [60]:
np.unique(y)

array([0.0000e+00, 1.0000e+00, 2.0000e+00, 2.0001e+04])

In [61]:
y.isna().sum()

0

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [63]:
del X
del y

In [64]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [65]:
del X_train
del y_train

In [66]:
from sklearn.metrics import accuracy_score

In [67]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [68]:
print(f'Model pontossága: {accuracy}')

Model pontossága: 0.9763564736221442


In [69]:
del y_pred
del y_test
del X_test

In [70]:
merged_df['res']=model.predict(merged_df[new_input])

In [71]:
merged_df['res'].value_counts()

res
2.0        1108949
1.0         467414
20001.0      18601
0.0           7789
Name: count, dtype: int64

In [72]:
merged_df['CONTRACT_CREDIT_INTERMEDIARY'].value_counts()

CONTRACT_CREDIT_INTERMEDIARY
2.0        1069644
1.0         454073
20001.0      18654
0.0            960
Name: count, dtype: int64

In [73]:
# merged_df=merged_df.drop('CONTRACT_TYPE_OF_INTEREST_REPAYMENT', axis=1)
merged_df=merged_df.drop('CONTRACT_CREDIT_INTERMEDIARY_0.0', axis=1)
merged_df=merged_df.drop('CONTRACT_CREDIT_INTERMEDIARY_1.0', axis=1)
merged_df=merged_df.drop('CONTRACT_CREDIT_INTERMEDIARY_2.0', axis=1)
merged_df=merged_df.drop('CONTRACT_CREDIT_INTERMEDIARY_20001.0', axis=1)


In [74]:
merged_df['CONTRACT_CREDIT_INTERMEDIARY'] = merged_df.apply(lambda row: row['res'] if pd.isna(row['CONTRACT_CREDIT_INTERMEDIARY']) else row['CONTRACT_CREDIT_INTERMEDIARY'], axis=1)
merged_df=merged_df.drop('res', axis=1)

In [75]:
dummy_cols=[]

def dummy_creator(df, column, dummy_col):

  ps_dummies = pd.get_dummies(df[column], prefix = column, drop_first = False)


  dummy_col.append(ps_dummies.columns)

  df = df.merge(ps_dummies, left_index = True, right_index = True, how = 'left')

  return df, dummy_col

In [76]:
# merged_df.rename(columns={'res': 'CONTRACT_TYPE_OF_INTEREST_REPAYMENT'}, inplace=True)

In [77]:
merged_df['CONTRACT_CREDIT_INTERMEDIARY'].value_counts()

CONTRACT_CREDIT_INTERMEDIARY
2.0        1107192
1.0         469118
20001.0      18654
0.0           7789
Name: count, dtype: int64

In [78]:
merged_df['CONTRACT_CREDIT_INTERMEDIARY'].isna().sum()

0

In [79]:
merged_df, dummy_cols=dummy_creator(merged_df,'CONTRACT_CREDIT_INTERMEDIARY', dummy_cols)
merged_df=merged_df.drop('CONTRACT_CREDIT_INTERMEDIARY', axis=1)

In [80]:
merged_df.to_csv('data/df_dummy_prepped-2-3.csv', index=False)

In [81]:
merged_df

,CONTRACT_ID,BORROWER_ID,CONTRACT_CREDIT_LOSS,CONTRACT_DATE_OF_LOAN_AGREEMENT,CONTRACT_DEPT_SERVICE_TO_INCOME,CONTRACT_INCOME,CONTRACT_INSTALMENT_AMOUNT,CONTRACT_INTEREST_RATE,CONTRACT_LGD,CONTRACT_LOAN_AMOUNT,...,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100003.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_100004.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_110001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140001.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140002.0,CONTRACT_TYPE_OF_INTEREST_REPAYMENT_140003.0,CONTRACT_CREDIT_INTERMEDIARY_0.0,CONTRACT_CREDIT_INTERMEDIARY_1.0,CONTRACT_CREDIT_INTERMEDIARY_2.0,CONTRACT_CREDIT_INTERMEDIARY_20001.0
0,TpK8osXs,d8SqtuEV,0.0,28,0.00,0.0,9545.0,22.08,175424.0,192161,...,False,False,False,False,False,False,False,False,True,False
1,EtIEHrcH,lrdxML0g,0.0,12,0.00,0.0,9528.0,16.57,994868.0,55590,...,False,False,False,False,False,False,False,False,True,False
2,1G10DfKj,gII7nnq4,16350.0,19,7.05,127305.0,8899.0,22.35,282553.0,275924,...,True,False,False,False,False,False,False,False,True,False
3,2NLT774,MMkJ8z/e,0.0,14,0.00,0.0,14329.0,0.00,68981.0,138862,...,False,False,False,False,False,False,False,False,True,False
4,VpylRvay,M417onFP,2395.0,67,0.00,0.0,4003.0,23.92,163319.0,169624,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1602748,5B25613A,Z9fieAgO,0.0,714,0.00,0.0,0.0,0.00,0.0,2853835,...,False,False,False,False,False,False,False,False,True,False
1602749,947A5B38,26ZlUKOa,0.0,499,0.00,0.0,0.0,0.00,0.0,80914,...,False,False,False,False,False,False,False,False,True,False
1602750,947A5B38,Dxng1cGs,0.0,499,0.00,0.0,0.0,0.00,0.0,80914,...,False,False,False,False,False,False,True,False,False,False
1602751,46115AEC,Ggu/Vok7,0.0,383,0.00,0.0,0.0,0.00,0.0,497293,...,False,False,False,False,False,False,False,False,True,False
